In [1]:
import pandas as pd
import json

In [2]:
def search_variables_definition(data_definitions, column_to_search):
    response = None
    is_finish=1
    for element in data_definitions:
        for key in element:
            if key == column_to_search:
                response = element[key]
                is_finish = 0
                break
        if is_finish == 0:
            break
    
    return response

In [3]:
def create_df_from_dict(dict_data):
    matrix_data = []
    
    for element in dict_data:
        row = [int(element['nominal_value']), element['description']]
        matrix_data.append(row)
    df_desc = pd.DataFrame(matrix_data, columns=['value', 'description'])
    return df_desc

In [4]:
def replace_values_for_description(df_values, array_data):
    array_data_replace = []

    for i in range(len(array_data)):

        try:
            df_filter = df_values.loc[df_values['value'] == int(array_data[i])]
            df_filter = df_filter.reset_index()

            array_data_replace.append(df_filter['description'][0])
        except:
            array_data_replace.append(None)
            
    return array_data_replace

In [5]:
df_data = pd.read_csv("../raw_data/data/raw_data.csv")

with open("../raw_data/data/desc_columns.json", 'r') as doc_open:
    definitions_variables_doc = json.load(doc_open)

In [6]:
relevant_columns = []

for column in df_data.columns:
    if ("ADS" in column.upper() or "SSP" in column.upper()) and "2" not in column:
        relevant_columns.append(column)

#adding relevant columns:
other_columns = ["change_attachment", "sensitivity2", "cis_total_mean", "Rango_ITERS", "insecure_bebe2", "Género_bebé", "Grupo_étnico_bebé",  "id_name", 'N°_diada']
relevant_columns += other_columns

columns_with_measures = ['id_name','m1','m2','m3','m4']

df_data_measures = df_data[columns_with_measures]
df_data_measures = df_data_measures.fillna(df_data_measures.mean(numeric_only=True))

df_data_features = df_data[relevant_columns]

In [7]:
dict_responses = {}
columns_without_def = []

for column in df_data_features.columns:
    if column not in ["id_name", "N°_diada"]:
        response = search_variables_definition(definitions_variables_doc, column)
        dict_responses.update({column:response})
        if response == None:
            columns_without_def.append(column)

filter_columns = [column for column in relevant_columns if column not in columns_without_def]

df_data_features_process = df_data_features[filter_columns]

In [8]:
df_data_features_update_values = pd.DataFrame()
df_data_features_update_values['id_name'] = df_data_features_process['id_name']
df_data_features_update_values["N°_diada"] = df_data_features_process["N°_diada"]

for column in df_data_features_process:
    if column not in ["id_name", "N°_diada"]:
        dict_value = dict_responses[column]
        df_description = create_df_from_dict(dict_value)
        df_data_features_update_values[column] = replace_values_for_description(df_description, df_data_features_process[column])
        
df_data_features_update_values.head(5)

,id_name,N°_diada,ADS_CHILD_M,ADS_CHILD_V,ADS_CHILD_TA,ADS_CHILD_SA,ADS_CHILD_A,ADS_CHILD_P,ADS_MAIN_CAREGIVER_M,ADS_MAIN_CAREGIVER_V,...,ADS_MAIN_CAREGIVER_A,ADS_MAIN_CAREGIVER_P,Attachment_ads,apego_SSP,Apego_dic_ADS,Apego_dic_SSP,change_attachment,Rango_ITERS,Género_bebé,Grupo_étnico_bebé
0,Maximiliano-Cárdenas-Catelicán,1,rara vez mira la cara de la madre y/o miradas...,rara vez vocaliza y/o lloriquea,rara vez toca y/o tiende hacia la madre,no se relaja em brazos de la madre y/o postura...,ansiedad moderada y/o expresiones ocasionales ...,rara vez sigue a la madre con el cuerpo o la m...,"ocasionalmente mira a la cara del bebe, altern...","palabras, vocalizaiones y/o sonidos escasos",...,"frecuentemente pasiva, apatica y/o irritada",intermitentemente alejada del bebe con acercam...,insecure,Inseguro ambivalente,insecure,insecure,no change,Calidad Media,Masculino,Mapuche
1,Martín-Pérez-González,4,rara vez mira la cara de la madre y/o miradas...,frecuentes vocalizaciones y/o llanto intenso,nunca toca y/o tiende hacia la madre,se resiste violentamente y/o se arquea alejand...,"tension ocasional, en general sonrie",nunca sigue a la madre con el cuerpo o la mirada,"ocasionalmente mira a la cara del bebe, altern...","palabras, vocalizaciones y/o sondos ocasionales",...,ansiedad moderada y/o expresiones ocasionales ...,intermitentemente alejada del bebe con acercam...,insecure,Seguro,insecure,secure,positive,Calidad Media,Masculino,Sin etnia
2,Noemí-Cárdenas-Gutierrez,5,rara vez mira la cara de la madre y/o miradas...,ocasionalmente vocaliza y/o llanto breve,ocasionalmente toca y/o tiende hacia la madre,se relaja en brazos de la madre y/o moldea su ...,ansiedad moderada y/o expresiones ocasionales ...,rara vez sigue a la madre con el cuerpo o la m...,"frecuentes miradas, laras y cortas a la cara d...","frecuentes palabras, vocalizaciones y/o sonidos",...,ansiedad moderada y/o expresiones ocasionales ...,contacto fisico frecuente con el bebe,secure,Seguro,insecure,secure,positive,Calidad Media,Femenino,Sin etnia
3,Jasna-Mansilla-Mansilla,7,"ocasionalmente mira a la cara de la madre, alt...",ocasionalmente vocaliza y/o llanto breve,rara vez toca y/o tiende hacia la madre,no se relaja em brazos de la madre y/o postura...,ansiedad moderada y/o expresiones ocasionales ...,rara vez sigue a la madre con el cuerpo o la m...,"ocasionalmente mira a la cara del bebe, altern...","palabras, vocalizaiones y/o sonidos escasos",...,"frecuentemente pasiva, apatica y/o irritada",intermitentemente alejada del bebe con acercam...,insecure,Inseguro evitante,insecure,insecure,no change,Calidad Media,Femenino,Sin etnia
4,Jhampiere-Largacha-Yañez,11,"ocasionalmente mira a la cara de la madre, alt...",ocasionalmente vocaliza y/o llanto breve,rara vez toca y/o tiende hacia la madre,no se relaja em brazos de la madre y/o postura...,ansiedad moderada y/o expresiones ocasionales ...,intermitentemente sigue a la madre con el cuer...,"frecuentes miradas, laras y cortas a la cara d...","frecuentes palabras, vocalizaciones y/o sonidos",...,"tension ocasional, en general sonrie",intermitentemente alejada del bebe con acercam...,secure,None,insecure,None,None,Calidad Media,Masculino,Sin etnia


In [9]:
pending_features = df_data_features[["change_attachment", "sensitivity2", "cis_total_mean", "Rango_ITERS", "insecure_bebe2", "Género_bebé", "Grupo_étnico_bebé",  "id_name"]]

In [10]:
df_data_to_export = pd.merge(df_data_features_update_values, df_data_measures, on="id_name")
df_data_to_export.head()

,id_name,N°_diada,ADS_CHILD_M,ADS_CHILD_V,ADS_CHILD_TA,ADS_CHILD_SA,ADS_CHILD_A,ADS_CHILD_P,ADS_MAIN_CAREGIVER_M,ADS_MAIN_CAREGIVER_V,...,Apego_dic_ADS,Apego_dic_SSP,change_attachment,Rango_ITERS,Género_bebé,Grupo_étnico_bebé,m1,m2,m3,m4
0,Maximiliano-Cárdenas-Catelicán,1,rara vez mira la cara de la madre y/o miradas...,rara vez vocaliza y/o lloriquea,rara vez toca y/o tiende hacia la madre,no se relaja em brazos de la madre y/o postura...,ansiedad moderada y/o expresiones ocasionales ...,rara vez sigue a la madre con el cuerpo o la m...,"ocasionalmente mira a la cara del bebe, altern...","palabras, vocalizaiones y/o sonidos escasos",...,insecure,insecure,no change,Calidad Media,Masculino,Mapuche,20.076728,31.719207,23.453752,25.016903
1,Martín-Pérez-González,4,rara vez mira la cara de la madre y/o miradas...,frecuentes vocalizaciones y/o llanto intenso,nunca toca y/o tiende hacia la madre,se resiste violentamente y/o se arquea alejand...,"tension ocasional, en general sonrie",nunca sigue a la madre con el cuerpo o la mirada,"ocasionalmente mira a la cara del bebe, altern...","palabras, vocalizaciones y/o sondos ocasionales",...,insecure,secure,positive,Calidad Media,Masculino,Sin etnia,28.584920,82.400761,28.361209,19.363161
2,Noemí-Cárdenas-Gutierrez,5,rara vez mira la cara de la madre y/o miradas...,ocasionalmente vocaliza y/o llanto breve,ocasionalmente toca y/o tiende hacia la madre,se relaja en brazos de la madre y/o moldea su ...,ansiedad moderada y/o expresiones ocasionales ...,rara vez sigue a la madre con el cuerpo o la m...,"frecuentes miradas, laras y cortas a la cara d...","frecuentes palabras, vocalizaciones y/o sonidos",...,insecure,secure,positive,Calidad Media,Femenino,Sin etnia,161.424693,46.867951,39.006348,52.728307
3,Jasna-Mansilla-Mansilla,7,"ocasionalmente mira a la cara de la madre, alt...",ocasionalmente vocaliza y/o llanto breve,rara vez toca y/o tiende hacia la madre,no se relaja em brazos de la madre y/o postura...,ansiedad moderada y/o expresiones ocasionales ...,rara vez sigue a la madre con el cuerpo o la m...,"ocasionalmente mira a la cara del bebe, altern...","palabras, vocalizaiones y/o sonidos escasos",...,insecure,insecure,no change,Calidad Media,Femenino,Sin etnia,26.747838,47.213491,21.599136,35.942378
4,Jhampiere-Largacha-Yañez,11,"ocasionalmente mira a la cara de la madre, alt...",ocasionalmente vocaliza y/o llanto breve,rara vez toca y/o tiende hacia la madre,no se relaja em brazos de la madre y/o postura...,ansiedad moderada y/o expresiones ocasionales ...,intermitentemente sigue a la madre con el cuer...,"frecuentes miradas, laras y cortas a la cara d...","frecuentes palabras, vocalizaciones y/o sonidos",...,insecure,None,None,Calidad Media,Masculino,Sin etnia,118.901337,47.213491,37.125222,382.715268


In [11]:
df_data_to_export = df_data_to_export.merge(right=pending_features, on="id_name")

In [12]:
df_data_to_export.columns

Index(['id_name', 'N°_diada', 'ADS_CHILD_M', 'ADS_CHILD_V', 'ADS_CHILD_TA',
       'ADS_CHILD_SA', 'ADS_CHILD_A', 'ADS_CHILD_P', 'ADS_MAIN_CAREGIVER_M',
       'ADS_MAIN_CAREGIVER_V', 'ADS_MAIN_CAREGIVER_TA',
       'ADS_MAIN_CAREGIVER_SA', 'ADS_MAIN_CAREGIVER_A', 'ADS_MAIN_CAREGIVER_P',
       'Attachment_ads', 'apego_SSP', 'Apego_dic_ADS', 'Apego_dic_SSP',
       'change_attachment_x', 'Rango_ITERS_x', 'Género_bebé_x',
       'Grupo_étnico_bebé_x', 'm1', 'm2', 'm3', 'm4', 'change_attachment_y',
       'sensitivity2', 'cis_total_mean', 'Rango_ITERS_y', 'insecure_bebe2',
       'Género_bebé_y', 'Grupo_étnico_bebé_y'],
      dtype='object')

In [13]:
df_data_to_export = df_data_to_export[['N°_diada', 'Apego_dic_SSP', 'Rango_ITERS_x', 'Género_bebé_x', 'm1', 'm2', 'm3', 'm4', 'sensitivity2', 'cis_total_mean']]

In [14]:
df_data_to_export.columns = ['ID', 'Attachment', 'ITERS range', 
                             'Gender', 'IgA D1-Morning', 'IgA D1-Afternoon', 'IgA D2-Morning', 
                             'IgA D2-Afternoon', 'Sensitivity', 'Cis total mean']

In [15]:
df_data_to_export.shape

(35, 10)

In [16]:
df_data_to_export.head(5)

,ID,Attachment,ITERS range,Gender,IgA D1-Morning,IgA D1-Afternoon,IgA D2-Morning,IgA D2-Afternoon,Sensitivity,Cis total mean
0,1,insecure,Calidad Media,Masculino,20.076728,31.719207,23.453752,25.016903,2,3.884615
1,4,secure,Calidad Media,Masculino,28.584920,82.400761,28.361209,19.363161,3,3.807692
2,5,secure,Calidad Media,Femenino,161.424693,46.867951,39.006348,52.728307,5,3.807692
3,7,insecure,Calidad Media,Femenino,26.747838,47.213491,21.599136,35.942378,1,3.846154
4,11,None,Calidad Media,Masculino,118.901337,47.213491,37.125222,382.715268,5,3.961538


In [17]:
df_data_to_export["Average IgA Morning"] = df_data_to_export[['IgA D1-Morning', 'IgA D2-Morning']].mean(axis=1)
df_data_to_export["Average IgA Afternoon"] = df_data_to_export[['IgA D1-Afternoon', 'IgA D2-Afternoon']].mean(axis=1)

df_data_to_export["Average Difference IgA"] = df_data_to_export["Average IgA Afternoon"]-df_data_to_export["Average IgA Morning"]


In [18]:
df_data_to_export.columns

Index(['ID', 'Attachment', 'ITERS range', 'Gender', 'IgA D1-Morning',
       'IgA D1-Afternoon', 'IgA D2-Morning', 'IgA D2-Afternoon', 'Sensitivity',
       'Cis total mean', 'Average IgA Morning', 'Average IgA Afternoon',
       'Average Difference IgA'],
      dtype='object')

In [19]:
df_data_to_export["Attachment"].value_counts()

secure      18
insecure     9
Name: Attachment, dtype: int64

In [20]:
df_data_to_export["Attachment"] = df_data_to_export["Attachment"].replace({"secure": "Secure", "insecure": "Insecure"})

In [21]:
df_data_to_export["Gender"].value_counts()

Femenino     20
Masculino    15
Name: Gender, dtype: int64

In [22]:
df_data_to_export["Gender"] = df_data_to_export["Gender"].replace({"Femenino": "Female", "Masculino": "Male"})

In [23]:
df_data_to_export["ITERS range"].value_counts()

Calidad Media    27
Baja Calidad      8
Name: ITERS range, dtype: int64

In [24]:
df_data_to_export["ITERS range"] = df_data_to_export["ITERS range"].replace({"Calidad Media": "Average Quality", "Baja Calidad": "Low Quality"})

In [26]:
df_data_to_export.columns

Index(['ID', 'Attachment', 'ITERS range', 'Gender', 'IgA D1-Morning',
       'IgA D1-Afternoon', 'IgA D2-Morning', 'IgA D2-Afternoon', 'Sensitivity',
       'Cis total mean', 'Average IgA Morning', 'Average IgA Afternoon',
       'Average Difference IgA'],
      dtype='object')

In [27]:
df_data_to_export = df_data_to_export.drop(columns=['ITERS range', 'Cis total mean','Sensitivity' ])

In [28]:
df_data_to_export.to_csv("../processed_data/1_processed_data.csv", index=False)